In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from zipfile import ZipFile
import cv2
import yaml
from tensorflow.keras.models import model_from_yaml
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout

In [52]:
batch_size = 5
 
''' 搭建模型'''
l=tf.keras.layers
model=Sequential()
 
#第一层卷积和池化
model.add(l.Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding='valid',input_shape=(227,227,3),activation='relu'))
model.add(l.BatchNormalization())
model.add(l.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))
 
#第二层卷积和池化
model.add(l.Conv2D(256,(5,5),(1,1),padding='same',activation='relu'))
model.add(l.BatchNormalization())
model.add(l.MaxPooling2D((3,3),(2,2),padding='valid'))
 
#第三层卷积
model.add(l.Conv2D(384,(3,3),(1,1),'same',activation='relu'))
 
#第四层卷积
model.add(l.Conv2D(384,(3,3),(1,1),'same',activation='relu'))
 
#第五层卷积和池化
model.add(l.Conv2D(256,(3,3),(1,1),'same',activation='relu'))
model.add(l.MaxPooling2D((3,3),(2,2),'valid'))
 
#全连接层
model.add(l.Flatten())
model.add(l.Dense(4096,activation='relu'))
model.add(l.Dropout(0.5))
 
model.add(l.Dense(4096,activation='relu'))
model.add(l.Dropout(0.5))
 
model.add(l.Dense(1000,activation='relu'))
model.add(l.Dropout(0.5))
 
#输出层
model.add(l.Dense(2,activation='softmax'))
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
# model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
'''导入图片数据'''

# ZipFile("/home/dzj/caofeng/image/image_train.zip","r").extractall("/home/dzj/caofeng/image/")
# ZipFile("/home/dzj/caofeng/image/image_test1.zip", "r").extractall("/home/dzj/caofeng/image/")

# image_train = "/home/dzj/caofeng/image/image_train"
# image_test = "/home/dzj/caofeng/image/image_test1"

#利用ImageDataGenerator生成一个batch一个batch的数据
datagen=ImageDataGenerator(samplewise_center=True,
                           rescale=1.0/255)   #samplewise_center:使输入数据的每个样本均值为0,rescale:归一化
 
train_generator = datagen.flow_from_directory(
    'D:/image/dataset/image_train',
    classes=['bhg','hg'],
    target_size=(227,227),
    class_mode='categorical',
    batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
    'D:/image/dataset/image_test1',
    classes=['bhg','hg'],
    target_size=(227,227),
    class_mode='categorical',
    batch_size=batch_size)

checkpoint_filepath = './AlexNet_model3.h5'
callback_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_accuracy', mode='max', save_best_only=True)
callback_learningrate = ReduceLROnPlateau(monitor='val_accuracy', mode='max', min_delta=0.03, patience=3, factor=.3, min_lr=0.000001, verbose=1)
callbacks = [callback_checkpoint,callback_learningrate]

'''开始训练'''
model.fit_generator(generator=train_generator,steps_per_epoch=210/5,epochs=30,validation_data=validation_generator,validation_steps=47/5,callbacks=callbacks)

yaml_string = model.to_yaml()  # 保存模型结构到yaml文件
open('./model_architecture3.yaml', 'w').write(yaml_string)
model.save_weights('./AlexNet_model3.h5')  #保存模型参数

Found 210 images belonging to 2 classes.
Found 47 images belonging to 2 classes.
Epoch 1/30
42/42 [==============================] - 27s 639ms/step - loss: 1.3155 - accuracy: 0.5429 - val_loss: 0.6576 - val_accuracy: 0.6809
Epoch 2/30
42/42 [==============================] - 27s 637ms/step - loss: 0.7172 - accuracy: 0.5714 - val_loss: 0.6424 - val_accuracy: 0.6809
Epoch 3/30
42/42 [==============================] - 26s 630ms/step - loss: 0.6917 - accuracy: 0.5952 - val_loss: 0.6628 - val_accuracy: 0.6809
Epoch 4/30
42/42 [==============================] - ETA: 0s - loss: 0.6667 - accuracy: 0.5905
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0029999999329447745.
42/42 [==============================] - 26s 631ms/step - loss: 0.6667 - accuracy: 0.5905 - val_loss: 0.6580 - val_accuracy: 0.6170
Epoch 5/30
42/42 [==============================] - 26s 626ms/step - loss: 0.5877 - accuracy: 0.6810 - val_loss: 0.6316 - val_accuracy: 0.6383
Epoch 6/30
42/42 [=======================

In [53]:
import yaml
import numpy as np
import cv2
import urllib.request
from tensorflow.keras.models import model_from_yaml
import matplotlib.pyplot as plt

model = model_from_yaml(open('./model_architecture3.yaml').read())
model.load_weights('./AlexNet_model3.h5')

In [58]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

imgs = []
image = url_to_image('https://img1.baidu.com/it/u=1138728463,1821870627&fm=26&fmt=auto&gp=0.jpg')
# plt.imshow(image)
img = cv2.resize(image,(256, 256))
imgs.append(img)
# plt.imshow(img)
a = np.array(imgs)
result=model.predict(a)
idx = np.argmax(result)

if idx == 0:
    print('the image is no\n')
else:
    print('the image is yes\n')

the image is yes



In [114]:
imgs=[]
img=cv2.imread('D:/image/微信图片_20210713161234.jpg')
img=cv2.resize(img,(227,227))
imgs.append(img)
a=np.array(imgs)
result=model.predict(a)
idx=np.argmax(result)
 
if idx==0:
    print('the image is hg\n')
else:
    print('the image is bhg\n')

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
